<a href="https://colab.research.google.com/github/pra-navv/MOM_generator_HuggingFace/blob/main/MinutesofMeeting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q requests torch bitsandbytes transformers sentencepiece accelerate openai httpx==0.27.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 70.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 10.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not curr

In [1]:
# imports

import os
import requests
from IPython.display import Markdown, display, update_display
from google.colab import drive
from huggingface_hub import login
from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TextStreamer, BitsAndBytesConfig, AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
import torch

In [2]:
# Constants

AUDIO_MODEL = "openai/whisper-medium"
LLAMA = "meta-llama/Meta-Llama-3.1-8B-Instruct"

In [3]:
# New capability - connect this Colab to my Google Drive
# See immediately below this for instructions to obtain denver_extract.mp3

drive.mount("/content/drive")
audio_filename = "/content/drive/MyDrive/Colab Notebooks/denver_extract.mp3"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Download denver_extract.mp3
You can either use the same file as me, the extract from Denver city council minutes, or you can try your own..

If you want to use the same as me, then please download my extract here, and put this on your Google Drive:
https://drive.google.com/file/d/1GRNmEG4C9zK9C75JNusRbJfFaFfxYI3g/view?usp=sharing

In [4]:
# Sign in to HuggingFace Hub

hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

In [5]:
speech_model = AutoModelForSpeechSeq2Seq.from_pretrained(AUDIO_MODEL, torch_dtype=torch.float16, low_cpu_mem_usage=True, use_safetensors=True)
speech_model.to('cuda')
processor = AutoProcessor.from_pretrained(AUDIO_MODEL)

pipe = pipeline(
    "automatic-speech-recognition",
    model=speech_model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch.float16,
    return_timestamps=True,
    device='cuda',
)

Device set to use cuda


In [6]:
# Use the Whisper OpenAI model to convert the Audio to Text
result = pipe(audio_filename)

/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


In [7]:
transcription = result["text"]
print(transcription)

 and kind of the confluence of this whole idea of the Confluence Week, the merging of two rivers. And as we've kind of seen recently in politics and in the world, there's a lot of situations where water is very important right now, and it's a very big issue. So that is the reason that the back of the logo is considered water. So let me see the creation of the logo here. So that basically kind of sums up the reason behind the logo and all the meanings behind symbolism and you'll hear a little bit more about our Confluence Week is basically highlighting all of these indigenous events and things that are happening around Denver so that we can kind of bring more people together and kind of share this whole idea of Indigenous People's Day. So thank you. Thank you so much and thanks for your leadership. All right. Welcome to the Denver City Council meeting of Monday, October 9th. rise with the Pledge of Allegiance by Councilman Lopez all right thank you Councilman Lopez, Madam Secretary roll

In [8]:
system_message = "You are an assistant that produces minutes of meetings from transcripts, with summary, key discussion points, takeaways and action items with owners, in markdown."
user_prompt = f"Below is an extract transcript of a Denver council meeting. Please write minutes in markdown, including a summary with attendees, location and date; discussion points; takeaways; and action items with owners.\n{transcription}"

messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]

In [9]:
#you can manually install BitsAndBytesConfig through pip:'!pip install bitsandbytes'

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

In [11]:
tokenizer = AutoTokenizer.from_pretrained(LLAMA)
tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
streamer = TextStreamer(tokenizer)
model = AutoModelForCausalLM.from_pretrained(LLAMA, device_map="auto", quantization_config=quant_config,resume_download=True)
outputs = model.generate(inputs, max_new_tokens=2000, streamer=streamer)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are an assistant that produces minutes of meetings from transcripts, with summary, key discussion points, takeaways and action items with owners, in markdown.<|eot_id|><|start_header_id|>user<|end_header_id|>

Below is an extract transcript of a Denver council meeting. Please write minutes in markdown, including a summary with attendees, location and date; discussion points; takeaways; and action items with owners.
 and kind of the confluence of this whole idea of the Confluence Week, the merging of two rivers. And as we've kind of seen recently in politics and in the world, there's a lot of situations where water is very important right now, and it's a very big issue. So that is the reason that the back of the logo is considered water. So let me see the creation of the logo here. So that basically kind of sums up the reason behind the logo and all the meaning

In [12]:
response = tokenizer.decode(outputs[0])

In [13]:
display(Markdown(response))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are an assistant that produces minutes of meetings from transcripts, with summary, key discussion points, takeaways and action items with owners, in markdown.<|eot_id|><|start_header_id|>user<|end_header_id|>

Below is an extract transcript of a Denver council meeting. Please write minutes in markdown, including a summary with attendees, location and date; discussion points; takeaways; and action items with owners.
 and kind of the confluence of this whole idea of the Confluence Week, the merging of two rivers. And as we've kind of seen recently in politics and in the world, there's a lot of situations where water is very important right now, and it's a very big issue. So that is the reason that the back of the logo is considered water. So let me see the creation of the logo here. So that basically kind of sums up the reason behind the logo and all the meanings behind symbolism and you'll hear a little bit more about our Confluence Week is basically highlighting all of these indigenous events and things that are happening around Denver so that we can kind of bring more people together and kind of share this whole idea of Indigenous People's Day. So thank you. Thank you so much and thanks for your leadership. All right. Welcome to the Denver City Council meeting of Monday, October 9th. rise with the Pledge of Allegiance by Councilman Lopez all right thank you Councilman Lopez, Madam Secretary roll call. Black. Clark. Here. Espinosa. Here. Flynn. Gilmour. Here. Here. Cashman. Here. Kenich. Here. Lopez. Here. New. Here. Ortega. Here. Sussman. Mr. President. Here. Eleven present. Eleven members present. We do have a quorum. Approval of the minutes. Are there any corrections to the minutes of October 2nd? Seeing none, minutes of October 2nd stand approved. Council Announcements. Are there any announcements by members of council? Councilman Clark. Thank you, Mr. President. I just wanted to invite everyone down to the first ever Halloween Parade on Broadway in Lucky District 7. It will happen on Saturday, October 21st At six o'clock p.m., it'll move along Broadway from Third to Alameda. It's gonna be a fun, family-friendly event. Everyone's invited to come down, wear a costume. There'll be candy for the kids, and there are tiki zombies and 29 hearses, and all kinds of fun and funky stuff on the fun and funky part of Broadway. So please join us October 21st at six o'clock for the Broadway Halloween. Brad, thank you, Mr. President. All right, thank you, Councilman Clark. I will be there. All right, presentations. Madam Secretary, do we have any presentations? None, Mr. President. Communications, do we have any communications? None, Mr. President. We do have one proclamation this evening, Proclamation 1127, an observance of the annual Indigenous Peoples Day in the city and county of Denver. Councilman Lopez, will you please read it? Thank you, Mr. President, with pride. Proclamation number 17, well let me just say this differently. Proclamation number 1127, series of 2017, an observance of the second annual Indigenous Peoples Day in the City and County of Denver. Whereas the Council of the City and County of Denver recognizes that the indigenous peoples have lived and flourished on the lands known as the Americas since time immemorial, and that Denver and the surrounding communities are built upon the ancestral homelands of numerous indigenous tribes, which include the Southern Ute, the Ute Mountain, Ute Tribes of Colorado. And whereas the tribal homelands and seasonal encampments of the Arapahoe and Cheyenne people along the banks of the Cherry Creek and South Platte River confluence give bearing to the future settlements that would become the birthplace of the Mile High City. And whereas Colorado encompasses the ancestral homelands of 48 tribes in the city and county of Denver and surrounding communities are home to the descendants of approximately 100 tribal nations. And whereas, on October 3rd, 2016, the City and County of Denver unanimously passed Council Bill 801, Series of 2016, officially designating the second Monday of October of each year as Indigenous Peoples' Day in Denver, Colorado. And whereas, the Council of the City and County of Denver continues to recognize and value the vast contributions made to the community through indigenous people's knowledge, science, philosophy, arts, and culture, and through these contributions, the city of Denver has developed and thrived. Whereas, the indigenous community, especially you, have made great efforts this year to draw attention to the contributions of indigenous people, including Confluence Week, Drawing record attendance to a national indigenous youth leadership conference, leading conversations on inclusion with their peers and supporting increased indigenous youth participation in science and engineering. Now therefore, be it proclaimed by this council of the city and county of Denver, section one, that the council of the city and county of Denver celebrates and honors the cultural and foundational contributions of indigenous people to our history, our past, our present and future. and continues to promote the education of the Denver community about these historic and contemporary contributions of indigenous people. Section two, that the City and County of Denver, Colorado does hereby observe October 9th, 2017 as Indigenous Peoples Day. Section three, that the clerk of the City and County of Denver shall attest and affix the seal of the City and County of Denver to this proclamation and that a copy be transmitted, excuse me, to the Denver American Indian Commission, the City and County of Denver School District number one, and the Colorado Commission on Indian Affairs. Thank you, Councilman Lopez. Your motion to adopt. Mr. President, I move that proclamation number 1127, series of 2017 be adopted. All right, it has been moved and seconded. It comes by members of council. Councilman Lopez. Thank you, Mr. President. It gives me a lot of pleasure and pride to read this proclamation officially for the third time, but as Indigenous Peoples' Day in Denver, officially for the second time. It's always awesome to be able to see, not just this proclamation come by my desk, but to see so many different people from our community in our council chambers. It was a very beautiful piece of artwork that you presented to us earlier and it is exactly the spirit that we drafted this proclamation and this actual, the ordinance that created Indigenous Peoples Day when we sat down and wrote it and as a community we couldn't think of anything else to begin except for the confluence of the two rivers and those confluence of the two rivers created such a great city. We live in such an amazing city and we're all proud of it and sometimes we, and a lot of people from all over the country or all over the world are proud of it, and sometimes a little too proud of it is telling them to go back home, but I'm kidding when I say that. But the really nice thing about this is that we are celebrating Indigenous Peoples' Day out of pride for who we are, who we are as a city and the contributions of indigenous people to the city, not out of spite, not out of a replacement of one culture over the other or out of contempt or disrespect. I think of a quote that Cesar Chavez made very popular and it stuck with me for a very long time, and anytime I have the opportunity to speak in front of children, and especially children in our community that, you know, they often second-guess themselves on where they're coming from, who they are, and I always say that, you know, it's very important to be proud of who you're from. And the quote that I use from Cesar Chavez is, you know, pride in one's own culture does does not require contempt or disrespect of another, right? And that's very important. It's very important for us to recognize that, no matter who we are, where we come from in this society, that your pride in your own culture does not require the contempt or disrespect of another. And man, what a year for that to just sit on our shoulders for a while, for us to think about, right? And so I wanted to thank you all, I think the commission. There's gonna be a couple individuals that are gonna come speak. Thank you for your art, your lovely artwork, for us to see what's in your heart and what now has become, probably is gonna be a very important symbol for the community. And also just for the work, the daily work every single day. We still have a lot of brothers and sisters whose ancestors once lived in these lands freely now stand on street corners, in poverty, without access to services, without access to sobriety, or even housing, or jobs. And what a cruel way to pay back a culture that has paved the way for the city to be built upon its shores. So we have a lot of work to do. And these kind of proclamations in this day is not a day off, but it's a day on in Denver, right? And addressing those critical issues. So, I know that my colleagues are very supportive. I'm gonna ask you to support this proclamation, as I know you always have done in the past. I'm very proud of today. Oh, and we made Time Magazine and Newsweek once again, today, as being a leader in terms of the cities that are celebrating Indigenous Peoples Day. I wanted to make a point out of that. Thank you, Councilman Lopez, and thank you for sponsoring this. Councilman Martega. Mr. President, I want to ask that my name be added. I don't think I could add much more to what Councilman Lopez has shared with us. I want to thank him for bringing this forward and really just appreciate all the contributions that our Native American community has contributed to this great city and great state. I worked in the Lieutenant Governor's Office when the Commission on Indian Affairs was created and had the benefit of being able to go down to the Four Corners for a peace treaty signing ceremony between the Utes and the Comanches that had been sort of at odds with each other for about 100 years and just being able to participate in that powwow was pretty awesome. So, and for those of you who continue to participate in the annual powwow, it's such a great opportunity for everybody else to enjoy so many of the contributions of the culture. I mean, to see that the dance continues to be carried on as well as the native language from generation to generation is just so incredible because in so many cultures, people have come here and assimilated to the norms here You know the norms here, and they lose their language and and and lose a lot of the culture and in the native community That that hasn't happened that has that you know Commitment to just passing that on from generation to generation is is so important and So I'm happy to be a co-sponsor of this tonight. Thank you Thank you. All right, thank you, Councilwoman Ortega. Councilwoman Kenich. Thank you very much, and I also wanna thank my colleague for bringing this forward. And I just wanted to say a word to the artists about how beautiful and moving I thought this logo was and your description of it. And I think one of the things that is clear is the words sometimes don't convey the power of imagery or music or the other pieces that make up culture, and so I think the art is so important. And when you talked about water, I was also thinking about land, and I guess I just wanted to say thank you. Many of the Native American peoples of Colorado have been at the forefront, or actually nationally, of defending some of the public lands that have been protected over the last few years that are under attack right now. And they're places that the communities have fought to protect, but that everyone gets to enjoy. And so I just think that it's an example of where cultural preservation intersects with environmental protection, with recreation, and all of the other ways that public lands are so important. And so I think I just wanted to say thank you for that, because I think we have some very sacred places in our country that are at risk right now. And so as we celebrate, I appreciate that there's still a piece of resistance in here. And I think that I just wanna mention a solidarity, and mention a feeling of solidarity with that resistance. So thank you, and happy Confluence Week. Thank you councilman can each and see no other comments. I'll just say a couple and in a time of such divisive Ugliness and just despicable behavior from our leadership The reason I'm so supportive of indigenous people's days because it means inclusivity It means respecting all respecting those who have been silenced on purpose for a long time and whose history has not been told, and so we celebrate inclusivity in the face of such evil times, honestly.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

**Meeting Minutes**
=====================


### Summary


*   **Attendees:** Denver City Council
*   **Location:** Denver City Council Chambers
*   **Date:** Monday, October 9th
*   **Summary:** The Denver City Council meeting celebrated Indigenous Peoples Day with a proclamation, highlighting the cultural and foundational contributions of indigenous people to the city's history, present, and future.


### Discussion Points


*   **Confluence Week:** The merging of two rivers, symbolizing the city's foundation and the importance of water in the community.
*   **Indigenous Peoples Day:** Recognizing the contributions of indigenous people to the city's history, present, and future.
*   **Proclamation 1127:** Officially designating October 9th as Indigenous Peoples Day in Denver, Colorado.
*   **Artwork:** A logo representing the confluence of two rivers, symbolizing the city's foundation and the importance of water in the community.


### Takeaways


*   **Pride in one's culture:** Does not require contempt or disrespect of another, as quoted by Cesar Chavez.
*   **Importance of art:** Conveys the power of imagery, music, and other pieces that make up culture.
*   **Cultural preservation:** Intersects with environmental protection, recreation, and other ways that public lands are important.
*   **Inclusivity:** Means respecting all, especially those who have been silenced or whose history has not been told.


### Action Items


*   **Adopt Proclamation 1127:** The Denver City Council adopted Proclamation 1127, officially designating October 9th as Indigenous Peoples Day in Denver, Colorado.
*   **Promote Education:** The city will continue to promote the education of the Denver community about the historic and contemporary contributions of indigenous people.
*   **Address Critical Issues:** The city will address issues affecting indigenous communities, such as poverty, lack of access to services, and housing.
*   **Celebrate Confluence Week:** The city will celebrate Confluence Week, highlighting the cultural and foundational contributions of indigenous people to the city's history, present, and future.


### Owners


*   **Councilman Lopez:** Sponsored the proclamation and led the discussion on Indigenous Peoples Day.
*   **Councilwoman Ortega:** Co-sponsored the proclamation and expressed her support for the celebration of Indigenous Peoples Day.
*   **Councilman Martega:** Co-sponsored the proclamation and shared his experiences with the Native American community.
*   **Councilwoman Kenich:** Expressed her support for the proclamation and highlighted the importance of cultural preservation and environmental protection.<|eot_id|>